# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [ ]:
!pip install torch==1.4.0
!pip install syft==0.2.9

In [2]:
import torch as th

x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [4]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

In [7]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [9]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
x = x.send(bob)

In [13]:
bob._objects

{82950212008: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

82950212008

In [16]:
x.id

67407047424

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker == x.owner

True

In [19]:
x

(Wrapper)>[PointerTensor | me:67407047424 -> bob:82950212008]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [22]:
alice = sy.VirtualWorker(hook, id="alice")

In [23]:
y = th.tensor([1, 2, 3, 4])
y_ptr = y.send(bob, alice)
y_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:37235671037 -> bob:49431400923]
	-> [PointerTensor | me:36106083555 -> alice:49431400923]

In [24]:
bob._objects

{49431400923: tensor([1, 2, 3, 4])}

In [25]:
alice._objects

{49431400923: tensor([1, 2, 3, 4])}

In [26]:
y_ptr.get()

[tensor([1, 2, 3, 4]), tensor([1, 2, 3, 4])]

In [27]:
bob._objects

{}

In [28]:
alice._objects

{}

In [29]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [30]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [31]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [32]:
x

(Wrapper)>[PointerTensor | me:66699303660 -> bob:47842494687]

In [33]:
y

(Wrapper)>[PointerTensor | me:9704806114 -> bob:72513750275]

In [34]:
z = x + y

In [35]:
z

(Wrapper)>[PointerTensor | me:49833589189 -> bob:39948235568]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:85484436048 -> bob:924957732]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [39]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [40]:
z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:31347391523 -> bob:54135348874]

In [41]:
z.backward()

(Wrapper)>[PointerTensor | me:44965176883 -> bob:21840368675]

In [42]:
x = x.get()

In [43]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [44]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [45]:
input = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad = True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad = True).send(bob)

In [46]:
weights= th.tensor([[0.], [0.]], requires_grad = True).send(bob)

In [47]:
for i in range(10):
  pred = input.mm(weights)

  loss = ((pred - target)**2).sum()

  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [48]:
bob = bob.clear_objects()

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{44196359214: tensor([1, 2, 3, 4, 5])}

In [52]:
del x

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
bob._objects

{52773040847: tensor([1, 2, 3, 4, 5])}

In [56]:
x = "asdf"

In [57]:
bob._objects

{}

In [58]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
x

(Wrapper)>[PointerTensor | me:49280752692 -> bob:24940866552]

In [60]:
bob._objects

{24940866552: tensor([1, 2, 3, 4, 5])}

In [61]:
x = "asdf"

In [62]:
bob._objects

{24940866552: tensor([1, 2, 3, 4, 5])}

In [63]:
del x

In [64]:
bob._objects

{24940866552: tensor([1, 2, 3, 4, 5])}

In [65]:
bob = bob.clear_objects()
bob._objects

{}

In [66]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [67]:
bob._objects

{47459689036: tensor([1, 2, 3, 4, 5])}

In [68]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [69]:
z = x + y

TensorsNotCollocatedException: ignored

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [70]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [71]:
from torch import nn, optim

In [72]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [73]:
# A Toy Model
model = nn.Linear(2,1)

In [74]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [75]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.4246)
tensor(0.7968)
tensor(0.5018)
tensor(0.3210)
tensor(0.2058)
tensor(0.1320)
tensor(0.0847)
tensor(0.0544)
tensor(0.0349)
tensor(0.0224)
tensor(0.0144)
tensor(0.0093)
tensor(0.0060)
tensor(0.0039)
tensor(0.0025)
tensor(0.0016)
tensor(0.0011)
tensor(0.0007)
tensor(0.0004)
tensor(0.0003)


In [76]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [77]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [78]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [79]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [80]:
train()

tensor(4.7028, requires_grad=True)
tensor(0.0860, requires_grad=True)
tensor(0.1048, requires_grad=True)
tensor(0.0996, requires_grad=True)
tensor(0.0441, requires_grad=True)
tensor(0.0733, requires_grad=True)
tensor(0.0314, requires_grad=True)
tensor(0.0529, requires_grad=True)
tensor(0.0232, requires_grad=True)
tensor(0.0383, requires_grad=True)
tensor(0.0173, requires_grad=True)
tensor(0.0278, requires_grad=True)
tensor(0.0129, requires_grad=True)
tensor(0.0203, requires_grad=True)
tensor(0.0097, requires_grad=True)
tensor(0.0149, requires_grad=True)
tensor(0.0072, requires_grad=True)
tensor(0.0110, requires_grad=True)
tensor(0.0054, requires_grad=True)
tensor(0.0081, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0060, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0044, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0033, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0013, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [81]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [82]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [83]:
x = x.send(alice)

In [84]:
bob._objects

{75699992781: tensor([1, 2, 3, 4, 5])}

In [85]:
alice._objects

{52399754409: (Wrapper)>[PointerTensor | alice:52399754409 -> bob:75699992781]}

In [86]:
y = x + x

In [87]:
y

(Wrapper)>[PointerTensor | me:48346691414 -> alice:38429563591]

In [88]:
bob._objects

{19250454058: tensor([ 2,  4,  6,  8, 10]),
 75699992781: tensor([1, 2, 3, 4, 5])}

In [89]:
alice._objects

{38429563591: (Wrapper)>[PointerTensor | alice:38429563591 -> bob:19250454058],
 52399754409: (Wrapper)>[PointerTensor | alice:52399754409 -> bob:75699992781]}

In [90]:
jon = sy.VirtualWorker(hook, id="jon")

In [91]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [92]:
bob._objects

{39831339630: tensor([1, 2, 3, 4, 5])}

In [93]:
alice._objects

{6637437244: (Wrapper)>[PointerTensor | alice:6637437244 -> bob:39831339630]}

In [94]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:6637437244 -> bob:39831339630]

In [95]:
bob._objects

{39831339630: tensor([1, 2, 3, 4, 5])}

In [96]:
alice._objects

{}

In [97]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [98]:
bob._objects

{}

In [99]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [100]:
bob._objects

{1409715842: tensor([1, 2, 3, 4, 5])}

In [101]:
alice._objects

{94436362892: (Wrapper)>[PointerTensor | alice:94436362892 -> bob:1409715842]}

In [102]:
del x

In [103]:
bob._objects

{}

In [104]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [105]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [106]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [107]:
bob._objects

{74775781579: tensor([1, 2, 3, 4, 5])}

In [108]:
alice._objects

{}

In [109]:
x.move(alice)

(Wrapper)>[PointerTensor | me:23569979133 -> alice:74775781579]

In [110]:
bob._objects

{}

In [111]:
alice._objects

{74775781579: tensor([1, 2, 3, 4, 5])}

In [112]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [113]:
bob._objects

{85118671477: tensor([1, 2, 3, 4, 5])}

In [114]:
alice._objects

{74775781579: tensor([1, 2, 3, 4, 5]),
 80013499230: (Wrapper)>[PointerTensor | alice:80013499230 -> bob:85118671477]}

In [115]:
x.remote_get()

(Wrapper)>[PointerTensor | me:58379377244 -> alice:80013499230]

In [116]:
bob._objects

{}

In [117]:
alice._objects

{74775781579: tensor([1, 2, 3, 4, 5]), 80013499230: tensor([1, 2, 3, 4, 5])}

In [118]:
x.move(bob)

(Wrapper)>[PointerTensor | me:58379377244 -> bob:80013499230]

In [119]:
x

(Wrapper)>[PointerTensor | me:58379377244 -> bob:80013499230]

In [120]:
bob._objects

{80013499230: tensor([1, 2, 3, 4, 5])}

In [121]:
alice._objects

{74775781579: tensor([1, 2, 3, 4, 5])}

# Section Project:

For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form

In [ ]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)